# LangChain全面剖析之Agents

## 1. 提出Agent架构设计理念的背景

吴恩达教授提出：当前，我们主要在零样本模式下使用 LLM，提供 prompt，逐个 token 地生成最终输出，没有进行调整。这类似于要求某人从头到尾写一篇文章，直接打字，不允许退格，并期望得到高质量的结果。尽管有困难，LLM 在这项任务上仍然表现得非常好！然而，通过智能体工作流，我们可以要求 LLM 多次迭代文档。例如，它可能需要执行一系列步骤：

- 规划大纲；

- 决定需要进行哪些网络搜索（如果需要），来收集更多信息；

- 写初稿；

- 通读初稿，找出不合理的论点或无关信息；

- 修改草稿；

......

这个迭代过程对于大多数人类作家写出好的文本至关重要。对于人工智能来说，这种迭代工作流会比单次编写产生更好的结果。

这是吴恩达教授提出的一个写作的例子，同时也映射了其他领域的工作流都可以这样去执行。那么对于`Agent`，我们期待它不仅仅是执行任务的工具，而是一个能够思考、自主分析需求、拆解任务并逐步实现目标的智能实体。这种形态的智能体才更接近于人工智能的终极目标——AGI（通用人工智能），它能让类似于托尼·斯塔克的贾维斯那样的智能助手成为现实，服务于每个人。

* 学习Model IO	：面向LLM开发
* 学习Chains		：面向Chains开发，多次调用大模型
* 学习Agent		：面向Agent开发，解决更复杂的问题
* 学习Lang Graph	：开发更高级的Agent

## 2. Chain模块设计的优劣势

### 2.1 介绍

大模型的应用绝不仅仅局限于简单的问答模式，一定会涉及多个复杂的中间环节。这其中

* 既需要考虑如何利用大模型的原生能力进行深度推理
* 又需要思考如何借助大模型调用外部工具的能力，用一种高效的方法将其串联成一条条业务流程，最终构建出一个完整的应用系统。

### 2.2 Chain模块使用回顾

我们回忆一下，在Chain模块的时候，如何完成的查询天气的业务。

它需要两个链路：

* 获取天气
* 解析天气



首先获取open AI key和base url

import openai
from openai import OpenAI
import os

openai.api_key=os.getenv("OPENAI_API_KEY")
openai.api_base="https://api.openai.com/v1"

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4",api_key=openai.api_key ,base_url=openai.api_base)

接下来定义外部工具get_weather函数，同时还配了一个干扰函数test来增加难度，检验大模型

In [3]:
import numpy as np
import pandas as pd

import json
import io
import inspect
import requests

In [4]:
open_weather_key = "5c939a7cc59eb8696f4cd77bf75c5a9a"

In [5]:
from langchain_core.tools import tool

import requests
import json

@tool
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询上海市天气，则loc参数需要输入'ShangHai'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

@tool
def test():
    """
    查询天气示例，这是一个测试
    """
    pass

In [7]:
weather_function_list = [get_weather, test]
weather_function_list

[StructuredTool(name='get_weather', description="get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询上海市天气，则loc参数需要输入'ShangHai'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息", args_schema=<class 'pydantic.v1.main.get_weatherSchema'>, func=<function get_weather at 0x7fa964585430>),
 StructuredTool(name='test', description='test() - 查询天气示例，这是一个测试', args_schema=<class 'pydantic.v1.main.testSchema'>, func=<function test at 0x7fa9645850d0>)]

准备第一个链路的提示词模版

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.openai_functions.base import create_openai_fn_chain
from langchain_openai import ChatOpenAI

# 构建第一个LLMChain的提示模版
weather_chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个能够查询天气的AI小助手"),
        ("human", "这是接收到的用户输入：{input}"),
    ]
)

准备第一个链路所需的输出解析器，用来提取大模型计算出的函数调用信息，并进行调用

In [9]:
from typing import List
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import BaseGenerationOutputParser
from langchain_core.outputs import ChatGeneration, Generation

class GetFunctonCallOutPutParser(BaseGenerationOutputParser[str]):
    """
    提取`create_openai_fn_chain`识别到的参数，执行对应的外部工具函数，并返回最终的结果
    """

    # 覆写parse_result方法
    def parse_result(self, result: List[Generation]):
        
        # 如果未识别到function_call关键字，返回原始文本
        if result[0].text:
            return result[0].text
        else:
            # 提取create_openai_fn_chain中拿到的参数 
            function_call = result[0].message.additional_kwargs["function_call"]

            # 提取到需要执行的具体函数名称： get_weather
            func_name = function_call["name"]
            
            # 使用全局定义的 function_list
            global weather_function_list

             # 格式化处理   
            func_map = {func.name: func for func in weather_function_list}
     
            # 提取经过create_openai_fn_runnable过程中识别到的函数名称
            chosen_tool = func_map[func_name]
    
    
            from operator import itemgetter
            # {'loc': 'Shanghai'}
            arguments = json.loads(itemgetter("arguments")(function_call))
   
            # 执行
            result = chosen_tool.invoke(arguments)
            return result

组装第一个链路

In [10]:
get_weather_data_chain = create_openai_fn_chain(llm=llm, 
                                                 prompt=weather_chat_template, 
                                                 functions=weather_function_list, 
                                                 output_parser=GetFunctonCallOutPutParser(),
                                                 output_key="weather_data",
                                                 verbose=True
                                                )

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `create_openai_fn_chain` was deprecated in LangChain 0.1.1 and will be removed in 0.2.0. Use create_openai_fn_runnable instead.
  warn_deprecated(


如上，`get_weather_data_chain`在执行后能够得到实时的天气数据。

接下来再定义一个`LLMChain`，用来接收该数据，并根据该数据情况，给出最终的出行建议。还是先定义输入模版。

In [11]:
from langchain_core.prompts import ChatPromptTemplate

analyse_weather_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位天气数据的分析大师"),
        ("human", "这是接收到的实时天气数据：{weather_data}"),
        ("human", "请你结合这份详细的数据，给予用户最合适的天气分析，并给出合理的出行建议")
    ]
)

构建第二个LLMChain，用来根据上一步得到的天气数据，进行分析并返回最终的结果。

In [12]:
from langchain.chains import LLMChain

analyse_weather_chains = LLMChain(llm=llm,
                                 prompt=analyse_weather_template,
                                 verbose=True
                                )

把两个LLMChain通过SimpleSequentialChain串联在一起

In [13]:
# 导入SimpleSequentialChain
from langchain.chains import SimpleSequentialChain

# 在chains参数中，按顺序传入LLMChain A 和LLMChain B
full_weather_chain = SimpleSequentialChain(chains=[get_weather_data_chain, analyse_weather_chains], 
                                           verbose=True,
                                           output_key="text",)

调用执行

In [15]:
full_weather_response = full_weather_chain.invoke("北京今天的天气怎么样？")
full_weather_response



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个能够查询天气的AI小助手
Human: 这是接收到的用户输入：北京今天的天气怎么样？

> Finished chain.
{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 20.94, "feels_like": 19.35, "temp_min": 20.94, "temp_max": 20.94, "pressure": 1008, "humidity": 10, "sea_level": 1008, "grnd_level": 1002}, "visibility": 10000, "wind": {"speed": 0.33, "deg": 196, "gust": 0.89}, "clouds": {"all": 100}, "dt": 1715824766, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715806718, "sunset": 1715858588}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位天气数据的分析大师
Human: 这是接收到的实时天气数据：{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\u59

{'input': '北京今天的天气怎么样？',
 'text': '根据提供的天气数据，现在北京的天气情况是多云，气温在20.94摄氏度，体感温度19.35摄氏度，湿度为10%，风速为0.33米/秒，风向196度，能见度为10000米。\n\n这是一个相对温暖的一天，但湿度较低，可能会觉得有些干燥。风速较轻，不会对出行产生太大影响。能见度非常高，适合进行户外活动。\n\n建议出行时穿戴适宜的衣服，以保持舒适的体感温度。同时，由于湿度较低，记得随身携带保湿产品，如润唇膏或者保湿霜，以防皮肤干燥。此外，由于能见度高，如果有计划进行户外活动，今天是个好天气。但请注意避开最强烈的阳光时段，防止紫外线伤害。'}

### 2.3 总结

#### (1) 整个流程给我们的启示

如上所述，该过程反应了在特定场景下某个流程的实现方式。当然它也可以是一个公司报销流程、一个购买演唱会门票的过程等实际业务场景下的链路。核心在于其流程是固定的，即我们在构建之前，就能够明确每一步的输入和输出，以及如何将结果或数据传递至下一步。这种预设流程能够按照开发者的思路稳定执行所有步骤，仅在最终步骤返回处理结果。在此过程中，大模型在整个链路中起到的作用是：

1. 解析用户输入，以确定需执行的特定流程。
2. 当需调用外部工具时，从用户输入中提取关键信息。
3. 基于分析数据，输出最终结果。

#### (2) 面向过程编程

从上述流程上看，在`Chains`的抽象设计框架下，一条链路可以整合无限多个工具，利用大模型的能力去确定应调用哪个工具（`TOOL`）以及如何从用户输入（`Input`）中提取执行特定工具所需的关键信息（`PARSER`）。因此，使用`Chains`构建流程时，我们能够预设工具的使用顺序。其核心思想类似于传统的面向过程编程，只是将部分分析和处理流程自动化，交由大模型负责。可以说整个链路中大模型的使用相对占比并不是特别高，但考虑到当前大模型输出的不稳定性，这种设计方法非常巧妙，其优势就在于使用`Chains`构建链路能够增强稳定性，有效控制每个处理环节，极大降低错误发生的可能性。特别适用于相对固定的业务流程场景。

#### (3) 面向过程编程的问题 

从这个视角来看，`Chains`的构建本质上是将一系列操作硬编码在代码逻辑中。固而就会产生一系列的不灵活性，如下所示：

我们不再问“北京的天气怎么样”，而是稍微改了一下问题“今天北京和大理的温差有多大？“

先前的链路就无法适应

In [16]:
full_weather_response = full_weather_chain.invoke("今天北京和大理的温差有多大？")
full_weather_response



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个能够查询天气的AI小助手
Human: 这是接收到的用户输入：今天北京和大理的温差有多大？

> Finished chain.
{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 20.94, "feels_like": 19.35, "temp_min": 20.94, "temp_max": 20.94, "pressure": 1008, "humidity": 10, "sea_level": 1008, "grnd_level": 1002}, "visibility": 10000, "wind": {"speed": 0.33, "deg": 196, "gust": 0.89}, "clouds": {"all": 100}, "dt": 1715825681, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715806718, "sunset": 1715858588}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位天气数据的分析大师
Human: 这是接收到的实时天气数据：{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\

{'input': '今天北京和大理的温差有多大？',
 'text': '根据您提供的天气数据，现在是北京的天气状况如下：\n温度相对舒适，气温20.94摄氏度，体感温度为19.35摄氏度。天气虽然有些阴，但云层比较密集，所以阳光并不强烈。气压在正常范围内，湿度仅为10%，相对较低，空气比较干燥。风速非常小，仅为0.33，风向为196度，几乎感觉不到风。能见度高达10000米，非常清晰。\n根据这个分析，这将是一个相对温暖且干燥的天气，虽然可能会有些阴沉，但并无大风或降雨影响出行。\n出行建议如下：\n1. 由于天气较干，建议增加饮水量，保持身体的水分平衡。\n2. 由于温度适中，您不必穿太厚的衣服，但一定要记住带些保暖物品，如果晚上温度下降应对。\n3. 虽然天气阴沉，但紫外线仍然存在，若长时间在户外，仍需做好防晒措施。\n4. 天气状况良好，非常适合外出活动。您可以尽情享受这个季节的温暖和怡人的天气。\n请您根据个人情况，做好针对性的准备，祝您有一个愉快的一天！'}

&emsp;&emsp;针对提问“我想知道，现在这个时刻，北京和上海的温差有多大？”这样一条输入，按照人类的思维逻辑出发应该包括执行两个步骤：首先是分别获取北京和大理当前的天气信息，其次是对这两个数据进行对比分析，从而得出温差。在我们构建的链路流程中，虽然已实现查询天气的功能，但当前的实现并不支持依次查询两个城市的天气，并且对于温差对比分析，现有的`analyse_weather_template`提示模板并不适用。这主要体现了两个问题：一是链路流程未能依序处理多城市天气查询的需求，二是缺少适用于温差分析的模板。

In [18]:
full_weather_response = full_weather_chain.invoke("请你帮我查两次今天的北京天气，第二次用于校验，并返回你根据这两次查询对比下的分析结果。")
full_weather_response



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个能够查询天气的AI小助手
Human: 这是接收到的用户输入：请你帮我查两次今天的北京天气，第二次用于校验，并返回你根据这两次查询对比下的分析结果。

> Finished chain.
{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 20.94, "feels_like": 19.35, "temp_min": 20.94, "temp_max": 20.94, "pressure": 1008, "humidity": 10, "sea_level": 1008, "grnd_level": 1002}, "visibility": 10000, "wind": {"speed": 0.33, "deg": 196, "gust": 0.89}, "clouds": {"all": 100}, "dt": 1715825702, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715806718, "sunset": 1715858588}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位天气数据的分析大师
Human: 这是接收到的实时天气数据：{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", 

{'input': '请你帮我查两次今天的北京天气，第二次用于校验，并返回你根据这两次查询对比下的分析结果。',
 'text': '根据您提供的数据，当前的天气情况是在北京，经度为116.3972，纬度为39.9075。天气现象是多云，温度为20.94摄氏度，体感温度为19.35摄氏度，最低温度和最高温度都是20.94摄氏度。大气压强为1008hPa，湿度为10%，海平面的气压也是1008hPa，而地面的气压是1002hPa。\n\n能见度为10000米，风速为0.33米/秒，风向为196度，风阵风为0.89米/秒。云量为100%。\n\n根据这些数据，我们可以得出以下分析和建议：\n\n1. 温度适中，体感温度较为舒适，适合户外活动。\n2. 天气多云，没有降雨，可以进行各种户外活动。\n3. 湿度较低，体感可能会偏干燥，建议出行时带上润唇膏或者保湿喷雾，及时补充水分。\n4. 风力较小，没有大风天气，出行无需特别注意。\n5. 云量较大，可能会有阴天的感觉，但不影响视线，驾驶出行时注意安全。\n\n总的来说，当前的天气状况对出行并无大的影响，是一个适合出行的好天气。'}

从本质上分析，`Chains`的设计理念是一旦识别出用户意图，就按预设的流程顺序执行，这样的设计就势必会造成当前的这种现状。但这既是它的优势也是劣势。优势在于对于固定流程，`Chains`的执行过程是非常稳定的。而劣势在于，面对用户多样化的输入，一个输入可能需要多次函数调用，且所需执行的流程可能是不固定的，不必完全遵循既定逻辑就能完成任务。解决这种劣势的关键在于：需要一个可以智能分析用户的输入，然后按照用户的输入不断的去执行，直到完成用户的全部需求的大脑，这个大脑，就是`Agents`。一个更像人的大脑一样，具备自定分析处理能力的智能体。

因此，在LangChain框架中，当前端接收到的用户输入变得更加灵活，且后端处理流程非固定时，采用`Agents`来构建整个链路就会更加契合。`Agents`的核心作用在于自动解析用户的意图，针对每个分解后的子任务，判断是否需要调用外部函数、调用次数，如何管理、存储中间结果及传递这些中间结果等复杂流程，就是`Agents `核心在做的事情。

接下来，我们就通过优化`full_weather_chain`流程作为示例，深切体会和理解LangChain中`Agents`的设计思路和实现方式。

## 3. Agents模块的架构设计理念剖析

### 3.1 理解Agent执行过程

#### (1) 挑战

对用户的一条输入，我们将其拆分成若干子任务，依序执行每个子任务并最终汇总结果进行输出。这个过程看起来非常直接，但在借助大模型的能力并以应用的形式来构建时，会存在以下几个难点：
1. 如何将子任务进行有效拆分？
2. 如何做外部工具的调用机制？这包括：
   - 工具识别过程。
   - 工具执行过程。
3. 如果做对子任务结果的综合分析及最终结果的生成？

#### (2) 任务拆解

针对上述提到的三个难点，我们可以依次去考虑解决方案。首先，对于任务的拆分，我们认为：一个原生能力较强的大模型是完全具备任务拆解的能力，比如我们来测试`gpt-4`模型：

In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4",api_key=openai.api_key ,base_url=openai.api_base)
resonse = llm.invoke("如果我想得到今天的北京和大理在当前时间段的气温差，我应该怎么做？")

print(resonse.content)

1. 首先，你需要分别查询北京和大理当前的气温。你可以通过互联网上的天气预报网站，如中国天气网、weather.com等，或者通过手机上的天气应用软件查询。

2. 查询到两地的气温后，用北京的气温减去大理的气温，就得到了两地的气温差。如果你是想比较两地的气温来决定旅行的话，这个气温差可以作为参考。

请注意，由于气温可能会随着时间和天气的变化而变化，所以你获取的气温差只能代表查询时间的情况，如果你需要更准确的信息，可能需要多次查询。


#### (3) 执行子任务

当可以通过提示工程让大模型去拆解原始的输入为多个子任务后，接下来的关键就有两点：

1. 制定一个策略来依序执行这些子任务并收集结果。
2. 综合全部子任务的结果，进行最终结果的响应。

自然，实现上述过程的细节也就包含两点：

1. 需要明确子任务的执行顺序
2. 要确保各个步骤之间的信息正确传递，以便综合分析和生成最终结果

可以用如下伪代码来理解：

```python
# 初始化一个空列表来存储每个子任务的结果
results = []

# 定义子任务列表
sub_tasks = ['子任务1', '子任务2', '子任务3', ...]

# 遍历子任务列表，依次执行每个任务
for task in sub_tasks:
    # 执行子任务并获取结果
    result = execute_task(task)
    # 将结果添加到结果列表中
    results.append(result)

# 对收集到的结果进行综合分析
final_result = analyze_results(results)
```

#### (4) 期间的外部工具调用

最后，实现子任务调用外部工具的机制对我们来说并不陌生，前面频繁介绍过调用外部工具的各种方式。很多大模型只要给它绑定外部工具，且每个外部工具有比较详细的描述，它本身就具备非常好的识别能力。比如`gpt`模型的`Function Calling`过程，`Qwen`模型的`ReAct`过程，对于这个过程，我们需要优化的是：

1. 如何把外部工具的描述更好？
2. 问题拆解本身也会分成很多步骤，每一个问题解决的路径如何定义？

### 3.1 langchain Agent的设计

#### (1) Agent的组成

`Agent`由三个组件组成：

* 提示词模版（`Prompt Template`)
* 大模型（`LLM`)
* 输出解析器（`Output Parser`)

`Agent`的输入（message）主要来源于三个方面：
* `input`代表用户的原始输入。
* `Model Reponse`指的是模型对某一个子任务的响应输出。
* `History`则能携带上下文的信息。

`Agent`的输出：

* 连接到实际的工具库（`Toolkits`) 
* 需要调用哪些工具，将由经过`Agent`模块后拆分的子任务来决定。

#### (2) Agent执行工具的过程

而我们知道，大模型调用外部函数会分为两个过程：识别工具和实际执行。Agent也一样

识别工具过程：集Message -> Agent -> Toolkits这个流程，可以视作一种“静态”的执行流程。包括

* 将子任务拆解
* 根据这些子任务在工具库中找到相应的工具，提取工具名称及所需参数，

实际工具执行：则会交给`AgentExecutor`。

综上需要理解的是：在LangChain的`Agents`实际架构中，`Agent`的角色是接收输入并决定采取的操作，但它本身并不直接执行这些操作。这一任务是由`AgentExecutor`来完成的。

* 将`Agent`（决策大脑）与`AgentExecutor`（执行操作的Runtime）结合使用，才构成了完整的`Agents`（智能体）
* 其中`AgentExecutor`负责调用代理并执行指定的工具，以此来实现整个智能体的功能。

其实整个过程实现起来是非常复杂的。但幸运的是，这些复杂的处理逻辑LangChain框架已经实现了，封装在框架内就不需要我们去操心这些底层逻辑问题，这也正是我们选择使用框架来进行应用开发的本质原因。

所以接下来我们需要掌握的是：如何使用LangChain实现好的抽象模块，去组合出一个功能完整的智能体。

### 3.2 手写自定义Tool Calling Agent理解它的实现

#### (1) 介绍

LangChain框架提供了多种预构建的`Agents`可以直接使用，同时也支持灵活的接入自定义`Agents`。只要掌握了上一小节中介绍的构建`Agents`的基本理念，就完全能够利用LangChain框架中已经抽象化的工具和方法，遵循相同的逻辑来创建新的`Agents`。

为了理解Agent的实现原理，先不实用内置的实现，而是根据 LangChian `Agents`的这种设计架构，尝试按照LangChain的规范，接入一个自定义的`Agents`。

LangChain中Agents模块的类继承结构如下所示:

```python
BaseSingleActionAgent --> LLMSingleActionAgent
                          OpenAIFunctionsAgent
                          XMLAgent
                          Agent --> <name>Agent  # Examples: ZeroShotAgent, ChatAgent


BaseMultiActionAgent  --> OpenAIMultiFunctionsAgent
```

其对应的API文档位置如下

[https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.agents](https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.agents)

实现代码如下，各小节对应上一节《Agent的组成》中的各个组件

#### (2) 导入提示词模版（PromptTemplate模块）

为了既处理用户提问，又处理前几个步骤中收集到的来自各种工具的数据，先准备一个提示词模版。

它包括三部分数据：

* 背景信息：“你是一个非常厉害的AI助手，但不了解实时的一些知识”
* 用户的提问
* 先前Agent工具调用和相应工具输出的消息序列：放在名为`agent_scratchpad`类型是`MessagesPlaceholder`的占位符对象中

In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是一个非常厉害的AI助手，但不了解实时的一些知识",),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

在上一小节介绍构建`Agents`的难点时已经强调过：`Agents`之所以能够灵活的处理更复杂的应用，在于它可以拆分出多个子任务，顺序的去执行子任务，而下一个子任务如何与之前的任务构建关联关系，则可以通过上下文的形式进行传递，从而让下一个子任务判断：当前应该怎么执行。

这一过程，就可以借助LangChain已经实现好的`MessagesPlaceholder`模块来实现。

下面事`MessagesPlaceholder`提供的方法

```python
class MessagesPlaceholder(BaseMessagePromptTemplate):
    """假设变量已经是消息列表的提示模板。"""
    
    variable_name: str
    """用作消息的变量名称。"""

    optional: bool = False
    """是否为可选变量，默认为False。"""
    
    @classmethod
    def get_lc_namespace(cls) -> List[str]:
        """获取langchain对象的命名空间。"""
        return ["langchain", "prompts", "chat"]
    
    def format_messages(self, **kwargs: Any) -> List[BaseMessage]:
        """使用kwargs格式化消息。

        参数:
            **kwargs: 用于格式化的关键字参数。

        返回:
            BaseMessage的列表。
        """
    
    @property
    def input_variables(self) -> List[str]:
        """这个提示模板的输入变量。

        返回:
            输入变量名称的列表。
        """
        return [self.variable_name] if not self.optional else []   
```

#### (3) 构建工具库（Toolkits模块）

接下来准备`Toolkits`组件，这一过程只要定义外部工具，存放进`Toolkits`中即可。

如下代码所示：

In [75]:
from langchain_core.tools import tool

import requests
import json

@tool
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询上海市天气，则loc参数需要输入'ShangHai'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

In [50]:
get_weather("Beijing")

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\\u9634\\uff0c\\u591a\\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 22.94, "feels_like": 21.5, "temp_min": 22.94, "temp_max": 22.94, "pressure": 1007, "humidity": 8, "sea_level": 1007, "grnd_level": 1002}, "visibility": 10000, "wind": {"speed": 1.26, "deg": 198, "gust": 1.82}, "clouds": {"all": 97}, "dt": 1715829455, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715806718, "sunset": 1715858588}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

In [76]:
tools = [get_weather]
tools

[StructuredTool(name='get_weather', description="get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询上海市天气，则loc参数需要输入'ShangHai'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息", args_schema=<class 'pydantic.v1.main.get_weatherSchema'>, func=<function get_weather at 0x7fa95070a8b0>)]

#### (4) 将工具库绑定给大模型

准备一个LLM，将工具库绑定给它

In [74]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4",api_key=openai.api_key ,base_url=openai.api_base)

In [78]:
llm_with_tools = llm.bind_tools(tools)
llm_with_tools

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fa950194520>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fa950197c40>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': "get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询上海市天气，则loc参数需要输入'ShangHai'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息", 'parameters': {'type': 'object', 'properties': {'loc': {}}, 'required': ['loc']}}}]})

#### (5) 构造Agent实例（Agent模块：封装Agent策略）

我们只需将它们格式化为 `OpenAI` 可以识别的工具格式并将它们传递给大模型即可。

这一过程便可以通过链路（Chain，LCEL的管道符）来构建出最后完整的`Agent`。

In [79]:
# 用于格式化中间步骤（代理操作、工具输出对）以发送到模型的输入消息
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)

# 用于将输出消息转换为`Agent`操作/`Agent`完成的组件。
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"] # 被格式化的内容来自intermediate_steps
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

如上所示：在构建`Agent`实例时，还需要借助·format_to_openai_tool_messages`和 `OpenAIToolsAgentOutputParser()`

**`format_to_openai_tool_messages`**

* `format_to_openai_tool_messages`用于格式化中间步骤（代理操作、工具输出对）以发送到模型的输入消息的组件。
* 其中一个关键的概念是：`intermediate_steps`，它用来表示LLM到目前为止采取的步骤以及观察结果。

它们的代码如下

```python
def format_to_openai_tool_messages(
    intermediate_steps: Sequence[Tuple[AgentAction, str]],
) -> List[BaseMessage]:
    """将（AgentAction，工具输出）元组转换为FunctionMessages。

    参数:
        intermediate_steps: LLM到目前为止采取的步骤以及观察结果

    返回:
        发送给LLM的消息列表，以便进行下一次预测

    """
    messages = []
    for agent_action, observation in intermediate_steps:
        if isinstance(agent_action, OpenAIToolAgentAction):
            # 创建新消息列表，包括来自代理操作的消息日志和基于当前观察结果的工具消息
            new_messages = list(agent_action.message_log) + [
                _create_tool_message(agent_action, observation)
            ]
            # 将新消息添加到总消息列表中，避免重复
            messages.extend([new for new in new_messages if new not in messages])
        else:
            # 如果不是OpenAIToolAgentAction，则直接添加代理操作的日志作为AI消息
            messages.append(AIMessage(content=agent_action.log))
    return messages
```

**`OpenAIToolsAgentOutputParser`**

`OpenAIToolsAgentOutputParser`用于将输出消息转换为`Agent`操作/`Agent`完成的组件。

```python
class OpenAIToolsAgentOutputParser(MultiActionAgentOutputParser):
    """解析消息为代理操作/结束。

    旨在与OpenAI模型一起使用，因为它依赖于特定的tool_calls参数来传达使用哪些工具。

    如果传递了tool_calls参数，则使用该参数获取工具名称和工具输入。

    如果没有传递，则假定AIMessage是最终输出。
    """

    @property
    def _type(self) -> str:
        return "openai-tools-agent-output-parser"

    def parse_result(
        self, result: List[Generation], *, partial: bool = False
    ) -> Union[List[AgentAction], AgentFinish]:
        if not isinstance(result[0], ChatGeneration):
            raise ValueError("该输出解析器仅适用于ChatGeneration输出")
        message = result[0].message
        return parse_ai_message_to_openai_tool_action(message)

    def parse(self, text: str) -> Union[List[AgentAction], AgentFinish]:
        raise ValueError("只能解析消息")

```

#### (6) 构造AgentExecutor（AgentExecutor模块：处理用户请求）

经过上述步骤，就已经完整的构建了`message` -> `Agent` - > `Toolkits`这一“静态”流程，最终呈现的是一个`agent`实例。最后，我们要将整个过程转变成一个动态的Runtime，这就借助`AgentExecutor`模块来实现。

`AgentExecutor`模块作为`Agent`决策出来需要执行操作的实际执行单元，必要参数为

* `agnet`实例
* 外部工具库`Toolkits`，
* 用例控制日志输出的两个参数

其实例化代码如下：

In [87]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, 
                               verbose=True, 
                               return_intermediate_steps=True)

`AgentExecutor`会实际的执行`Agent`拆解出来的流程。

其中AgentExecutor的代码注释如下，详细内容可以在PyCharm中点击查看源代码来获得

```python
class AgentExecutor(Chain):
    """使用工具的代理执行器。"""

    agent: Union[BaseSingleActionAgent, BaseMultiActionAgent]
    """用于创建计划和决定每个执行循环步骤中的操作的代理。"""
    tools: Sequence[BaseTool]
    """代理可以调用的有效工具。"""
    return_intermediate_steps: bool = False
    """是否在最终输出之外返回代理的中间步骤轨迹。"""
    max_iterations: Optional[int] = 15
    """在结束执行循环之前要采取的最大步骤数。
    
    设置为'None'可能会导致无限循环。"""
    max_execution_time: Optional[float] = None
    """在执行循环中花费的最大墙钟时间。"""
    early_stopping_method: str = "force"
    """如果代理永远不返回`AgentFinish`时使用的提前停止方法。'force'或'generate'。

    `"force"` 返回一个字符串，说明它因为达到时间或迭代限制而停止。
    
    `"generate"` 在最后一次基于之前的步骤调用代理的LLM Chain来生成最终答案。
    """
    handle_parsing_errors: Union[
        bool, str, Callable[[OutputParserException], str]
    ] = False
    """如何处理代理的输出解析器引发的错误。
    默认为`False`，将抛出错误。
    如果为`true`，错误将被发送回LLM作为观察。
    如果是字符串，该字符串本身将作为观察发送给LLM。
    如果是可调用函数，将使用异常作为参数调用该函数，该函数的结果将作为观察传递给代理。
    """
    trim_intermediate_steps: Union[
        int, Callable[[List[Tuple[AgentAction, str]]], List[Tuple[AgentAction, str]]]
    ] = -1
    """修剪中间步骤的方式，可以是一个整数或一个函数，该函数接收中间步骤列表并返回修剪后的列表。"""
```

#### (7) 测试

最后，通过`AgentExecutor`实例来接收输入，启动`Agents`进行响应。

**先测试一个简单问题，查看Agent的任务拆解**

In [82]:
agent_executor.invoke({"input": "查询一下北京今天的天气"})



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'loc': 'BeiJing'}`


{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 22.94, "feels_like": 21.5, "temp_min": 22.94, "temp_max": 22.94, "pressure": 1007, "humidity": 8, "sea_level": 1007, "grnd_level": 1002}, "visibility": 10000, "wind": {"speed": 1.26, "deg": 198, "gust": 1.82}, "clouds": {"all": 97}, "dt": 1715829719, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715806718, "sunset": 1715858588}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}北京今天的天气情况为多云，当前的气温是22.94℃，感觉像21.5℃， 湿度为8%。风速为1.26m/s，风向为198°。

> Finished chain.


{'input': '查询一下北京今天的天气',
 'output': '北京今天的天气情况为多云，当前的气温是22.94℃，感觉像21.5℃， 湿度为8%。风速为1.26m/s，风向为198°。',
 'intermediate_steps': [(ToolAgentAction(tool='get_weather', tool_input={'loc': 'BeiJing'}, log="\nInvoking: `get_weather` with `{'loc': 'BeiJing'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_tvJvcwLgjOW1t2vQmu8Pzt4p', 'function': {'arguments': '{\n  "loc": "BeiJing"\n}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-74ee9c2e-8a1f-41ae-9f77-7a09bc06b1dc', tool_calls=[{'name': 'get_weather', 'args': {'loc': 'BeiJing'}, 'id': 'call_tvJvcwLgjOW1t2vQmu8Pzt4p'}], tool_call_chunks=[{'name': 'get_weather', 'args': '{\n  "loc": "BeiJing"\n}', 'id': 'call_tvJvcwLgjOW1t2vQmu8Pzt4p', 'index': 0}])], tool_call_id='call_tvJvcwLgjOW1t2vQmu8Pzt4p'),
   '{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\\u9634\\uff0c\\u591a\\u4

**用stream方法查看详细中间步骤**

可以通过`stream`方法查看到在`Agents`处理过程中所采取的中间步骤和的最终响应。执行方式如下：

In [83]:
list(agent_executor.stream({"input": "查询一下今天北京的天气？"}))



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'loc': 'Beijing'}`


{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 22.94, "feels_like": 21.5, "temp_min": 22.94, "temp_max": 22.94, "pressure": 1007, "humidity": 8, "sea_level": 1007, "grnd_level": 1002}, "visibility": 10000, "wind": {"speed": 1.26, "deg": 198, "gust": 1.82}, "clouds": {"all": 97}, "dt": 1715829455, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715806718, "sunset": 1715858588}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}今天北京的天气状况是阴，多云，温度为22.94摄氏度，感觉像是21.5摄氏度。最低温度为22.94摄氏度，最高温度为22.94摄氏度。相对湿度为8%，风速为1.26米/秒，风向为198度。

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_weather', tool_input={'loc': 'Beijing'}, log="\nInvoking: `get_weather` with `{'loc': 'Beijing'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_wEnPrYiIW6RYVV9MaycSwurp', 'function': {'arguments': '{\n  "loc": "Beijing"\n}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-74b5002a-05af-4935-bda0-393b7a860850', tool_calls=[{'name': 'get_weather', 'args': {'loc': 'Beijing'}, 'id': 'call_wEnPrYiIW6RYVV9MaycSwurp'}], tool_call_chunks=[{'name': 'get_weather', 'args': '{\n  "loc": "Beijing"\n}', 'id': 'call_wEnPrYiIW6RYVV9MaycSwurp', 'index': 0}])], tool_call_id='call_wEnPrYiIW6RYVV9MaycSwurp')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_wEnPrYiIW6RYVV9MaycSwurp', 'function': {'arguments': '{\n  "loc": "Beijing"\n}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metad

**再测试一个稍微复杂一点的问题，查看它的任务规划**

In [86]:
agent_executor.invoke({"input": "对比一下今天北京和大理的温差？"})



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'loc': 'BeiJing'}`


{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 22.94, "feels_like": 21.5, "temp_min": 22.94, "temp_max": 22.94, "pressure": 1007, "humidity": 8, "sea_level": 1007, "grnd_level": 1002}, "visibility": 10000, "wind": {"speed": 1.26, "deg": 198, "gust": 1.82}, "clouds": {"all": 97}, "dt": 1715829719, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715806718, "sunset": 1715858588}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}
Invoking: `get_weather` with `{'loc': 'DaLi'}`


{"coord": {"lon": 100.1833, "lat": 25.7}, "weather": [{"id": 802, "main": "Clouds", "description": "\u591a\u4e91", "icon": "03d"}], "base": "stations", "main": {"temp": 25.03, "feels_like": 24.74, "temp_min": 25.03, "temp_max": 25.03, "pressure": 1013, "humidity":

{'input': '对比一下今天北京和大理的温差？',
 'output': '北京现在的温度是22.94℃，而大理现在的温度是25.03℃，所以大理比北京温度高出2.09℃。',
 'intermediate_steps': [(ToolAgentAction(tool='get_weather', tool_input={'loc': 'BeiJing'}, log="\nInvoking: `get_weather` with `{'loc': 'BeiJing'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_CNzlrC9GPASLB9DElq8c9WFT', 'function': {'arguments': '{\n  "loc": "BeiJing"\n}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-817cf2b2-9627-4358-955b-c32c6b766b3b', tool_calls=[{'name': 'get_weather', 'args': {'loc': 'BeiJing'}, 'id': 'call_CNzlrC9GPASLB9DElq8c9WFT'}], tool_call_chunks=[{'name': 'get_weather', 'args': '{\n  "loc": "BeiJing"\n}', 'id': 'call_CNzlrC9GPASLB9DElq8c9WFT', 'index': 0}])], tool_call_id='call_CNzlrC9GPASLB9DElq8c9WFT'),
   '{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\\u9634\\uff0c\\u591a\\u4e91", "ic

在这个过程中，执行了两个`action`，大模型识别出了需要分别查询一次北京的天气数据和大理的天气数据后，才能结合数据回答用户`北京和上海，在现在这个季节温差大吗？`这样的需求。而整个过程，均有`Agents`自主完成。而功能实现至此，已经实现了一个基础的`Agents`，同时也借助这个过程给大家完善了LangChain在自定义`Agents`在构造过程中的核心的细节。

## 4. 使用LangChain内置模块快速实现Tool Calling Agent

### 4.1 文档

上面同样的功能，可以使用LangChain提供的内置模块来实现，代码更简单

[https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/) 

### 4.2 代码

#### (1) 导入依赖

In [88]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

#### (2) 下载来自langchainhub的提示词模版

直接下载已经写好的提示模版，注：这个过程需要开启科学上网。

In [90]:
! pip install langchainhub

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [91]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [92]:
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


#### (3) 准备外部工具库`Toolkits`

In [93]:
weather_function_list = [get_weather]

#### (4) 实例化模型和`Agent`

In [94]:
# # 构造`Agent`组件
agent = create_openai_tools_agent(llm, weather_function_list, prompt)

#### (5) 实例化 `AgentExecutor`

In [95]:
# 实例化 `AgentExecutor`
agent_executor = AgentExecutor(agent=agent, tools=weather_function_list)

#### (6) `Agents`功能测试

先测试一个简单点的问题

In [96]:
agent_executor.invoke(
    {
        "input": "今天的北京和上海的天气，现在相差多少度？"
    }
)

{'input': '今天的北京和上海的天气，现在相差多少度？',
 'output': '现在北京的温度是23.94度，上海的温度是24.92度，所以温度相差大约0.98度。'}

再测试一个复杂点的问题，体现它的任务规划

In [97]:
agent_executor.invoke(
    {
        "input": "今天的北京、天津、上海和广州，当前时间哪个城市的气温最低？"
    }
)

{'input': '今天的北京、天津、上海和广州，当前时间哪个城市的气温最低？',
 'output': '朋友，我查询到目前北京的气温是23.94°C，天津的气温是24.97°C。让我继续为你查询上海和广州的气温。'}

## 5. 手写自定义ReAct Agent理解它的实现

### 5.1 介绍

Agent有多种实现范式

* 前两小节的Tools Calling Agent是先规划拆解，再分步执行
* 这两小节的ReAct Agent则是“看一步走一步”，一边拆解子步骤（Reasoning）一边执行（Action）

同样先不用Langchain提供的内置组件，而是手写一个能够介入LangChain的自定义Agent来理解它的实现

### 5.2 代码

#### (1) 导入提示词模版

用来封装ReAct策略的提示词模版，它是核心

In [100]:
from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

这里面必须包含的四个变量是：
1. `input`：用户的输入
2. `agent_scratchpad`：中间的过程信息
3. `tools`：每个工具的描述
4. `tool_names`：每个工具的名称

翻译成中文，内容如下：

```txt
尽你所能回答以下问题。你可以使用以下工具：

{tools}

使用以下格式：

问题：你必须回答的输入问题
思考：你应该总是思考要做什么
行动：要采取的行动，应该是[{tool_names}]之一
行动输入：对行动的输入
观察：行动的结果
...（这个思考/行动/行动输入/观察可以重复N次）
思考：我现在知道最终答案
最终答案：原始输入问题的最终答案

开始！

问题：{input}
思考：{agent_scratchpad}
```

#### (2) 构建工具库

接下来构建工具库，这里仅使用`get_weather`函数。

In [98]:
from langchain_core.tools import tool

import requests
import json

@tool
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询上海市天气，则loc参数需要输入'ShangHai'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

In [99]:
get_weather("Beijing")

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\\u9634\\uff0c\\u591a\\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 24.94, "feels_like": 23.7, "temp_min": 24.94, "temp_max": 24.94, "pressure": 1006, "humidity": 8, "sea_level": 1006, "grnd_level": 1000}, "visibility": 10000, "wind": {"speed": 2.57, "deg": 195, "gust": 3.44}, "clouds": {"all": 95}, "dt": 1715833208, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715806718, "sunset": 1715858588}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

In [101]:
tools = [get_weather]
tools

[StructuredTool(name='get_weather', description="get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询上海市天气，则loc参数需要输入'ShangHai'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息", args_schema=<class 'pydantic.v1.main.get_weatherSchema'>, func=<function get_weather at 0x7fa9505cdc10>)]

#### (3) 将工具库绑定给大模型

**先提取工具描述和工具名称**

从函数的代码注释中提取，函数需要用@tool装饰器进行装饰

In [102]:
from langchain.tools.render import render_text_description

prompt = prompt.partial(
    tools=render_text_description(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
)

prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "get_weather: get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询上海市天气，则loc参数需要输入'ShangHai'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息", 'tool_names': 'get_weather'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\

**再将工具绑定给大模型**


In [103]:
from langchain_openai import ChatOpenAI

llm_with_stop = llm.bind(stop=["\nObservation"])

这里注意：我们之前也介绍过，当不适用OpenAI Tools规范时，不可再使用`bind_tools`方法进行绑定，尽管我们这里实例化的仍然是GPT模型，而是需要根据`ReAct`模版的规范去绑定并且指定每一步的操作，如果遇到`\nObservation`便进入下一轮的处理逻辑，直至完成最后一个子任务在输出最终的处理结果。

#### (4) 构造Agent实例

构造提示模版、实例化模型及将工具库与模型绑定后，接下来就可以去构建一个`Agent`实例

代码如下：

In [104]:
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str

output_parser = ReActSingleInputOutputParser()

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_stop
    | output_parser
)

这个`agent`的实例化过程，如果真正理解了`3. 如何自定义构建一个Agents`中我们介绍的流程，就一定能理解这个实例化`agent`时改变的参数。首先，`format_log_to_str`用于格式化地生成中间过程的关键数据，而output_parser，则需要按照ReAct的规范格式化每一个子任务的输出。

#### (5) 构造AgentExecutor实例

In [105]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

#### (6) 测试

In [111]:
agent_executor.invoke({"input": "北京和大理今天的气温差距多大？"})



> Entering new AgentExecutor chain...
我需要先分别查询北京和大理今天的气温，然后计算出他们的温度差。
Action: get_weather
Action Input: Beijing{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 24.94, "feels_like": 23.7, "temp_min": 24.94, "temp_max": 24.94, "pressure": 1004, "humidity": 8, "sea_level": 1004, "grnd_level": 999}, "visibility": 10000, "wind": {"speed": 2.64, "deg": 202, "gust": 3.64}, "clouds": {"all": 94}, "dt": 1715833801, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715806718, "sunset": 1715858588}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}我已经得到了北京的天气信息，当前的温度是24.94摄氏度。接下来我需要查询大理的天气信息。
Action: get_weather
Action Input: DaLi{"coord": {"lon": 100.1833, "lat": 25.7}, "weather": [{"id": 802, "main": "Clouds", "description": "\u591a\u4e91", "icon": "03d"}], "base": "stations", "main": {"temp": 25.68, "feels_like": 25.3, "temp_min": 25.

{'input': '北京和大理今天的气温差距多大？', 'output': '北京和大理今天的气温差距是0.74摄氏度。'}

In [116]:
agent_executor.invoke({"input": "北京、上海和广州，今天哪个城市的气温最低？"})



> Entering new AgentExecutor chain...
我需要分别获取北京、上海和广州的天气信息，然后比较它们的气温，找出最低的那个。
Action: get_weather
Action Input: BeiJing{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 24.94, "feels_like": 23.7, "temp_min": 24.94, "temp_max": 24.94, "pressure": 1004, "humidity": 8, "sea_level": 1004, "grnd_level": 999}, "visibility": 10000, "wind": {"speed": 2.64, "deg": 202, "gust": 3.64}, "clouds": {"all": 94}, "dt": 1715834411, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715806718, "sunset": 1715858588}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}我已经获取到了北京的天气信息，现在我需要获取上海和广州的天气信息。
Action: get_weather
Action Input: ShangHai{"coord": {"lon": 121.4581, "lat": 31.2222}, "weather": [{"id": 800, "main": "Clear", "description": "\u6674", "icon": "01d"}], "base": "stations", "main": {"temp": 25.92, "feels_like": 25.06, "temp_min": 25.92,

{'input': '北京、上海和广州，今天哪个城市的气温最低？', 'output': '今天北京的气温最低。'}

&emsp;&emsp;观察输出结果，可以看出，尽管整个核心流程仍然遵循LangChain定义的标准实现，但中间处理过程与我们之前使用OpenAI Tools实践的有显著不同。这一差异主要由ReAct模板控制，特别是每个`actions`中的`log`描述非常关键。对于四次查询天气的需求，`Agent`会根据上一步的处理结果来决定下一步的操作。然后，它会持续执行，直到认为已经获得了正确的答案。

&emsp;&emsp;这就是通过`ReAct`去构造`Agents`的过程，首先从构建流程上看其变化不大，只需要微调其中的几个细节，这包括修改提示模版，修改模型绑定工具库的方法和修改中间过程的格式化处理方法，所以当大家能够理解LangChain的`Agents`整体结构，再套用任何一个提示结构，甚至是自己去写提示信息来引导`Agents`的行为都是非常容易的。那么对于`ReAct`，LangChain也是同OpenAI Tools一样实现了封装，大家可以在这个入口去自行尝试：https://python.langchain.com/docs/modules/agents/agent_types/

## 6. 使用LangChain内置模块快速实现ReAct Agent

### 6.1 文档

上面同样的功能，可以使用LangChain提供的内置模块来实现，代码更简单

[https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/)

### 6.2 代码

#### (1) 构建工具库

In [131]:
from langchain_core.tools import tool

import requests
import json

@tool
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询上海市天气，则loc参数需要输入'ShangHai'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

In [132]:
get_weather("BeiJing")

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\\u9634\\uff0c\\u591a\\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 24.94, "feels_like": 23.7, "temp_min": 24.94, "temp_max": 24.94, "pressure": 1004, "humidity": 8, "sea_level": 1004, "grnd_level": 999}, "visibility": 10000, "wind": {"speed": 2.64, "deg": 202, "gust": 3.64}, "clouds": {"all": 94}, "dt": 1715834720, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715806718, "sunset": 1715858588}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

In [133]:
from langchain.agents import AgentExecutor, create_react_agent
tools = [get_weather]

#### (2) 导入提示词模版

In [134]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

#### (3) 构造Agent实例

In [135]:
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

#### (4) 构造AgentExecutor实例

In [136]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

#### (5) 测试

In [137]:
agent_executor.invoke({"input": "大理今天的天气怎么样？"})



> Entering new AgentExecutor chain...
我需要使用get_weather函数，将"DaLi"作为输入参数，以获取大理的天气信息。
Action: get_weather
Action Input: DaLi{"coord": {"lon": 100.1833, "lat": 25.7}, "weather": [{"id": 802, "main": "Clouds", "description": "\u591a\u4e91", "icon": "03d"}], "base": "stations", "main": {"temp": 25.68, "feels_like": 25.3, "temp_min": 25.68, "temp_max": 25.68, "pressure": 1010, "humidity": 38, "sea_level": 1010, "grnd_level": 810}, "visibility": 10000, "wind": {"speed": 2.21, "deg": 252, "gust": 3.89}, "clouds": {"all": 41}, "dt": 1715835406, "sys": {"country": "CN", "sunrise": 1715812399, "sunset": 1715860689}, "timezone": 28800, "id": 1814093, "name": "Dali", "cod": 200}根据观察到的结果，大理的当前天气情况是多云，并且温度很暖和，约在25.68摄氏度。风速为2.21米/秒，风向252度，能见度达到了10公里。因此我现在知道了最后的答案。
Final Answer: 大理今天的天气是多云的，温度约为25.68摄氏度，风速为2.21米/秒，风向252度，能见度为10公里。

> Finished chain.


{'input': '大理今天的天气怎么样？',
 'output': '大理今天的天气是多云的，温度约为25.68摄氏度，风速为2.21米/秒，风向252度，能见度为10公里。'}

In [138]:
agent_executor.invoke({"input": "大理和北京今天的温差多大？"})



> Entering new AgentExecutor chain...
我需要先查看一下大理和北京今天的具体温度。
Action: get_weather
Action Input: DaLi{"coord": {"lon": 100.1833, "lat": 25.7}, "weather": [{"id": 802, "main": "Clouds", "description": "\u591a\u4e91", "icon": "03d"}], "base": "stations", "main": {"temp": 25.68, "feels_like": 25.3, "temp_min": 25.68, "temp_max": 25.68, "pressure": 1010, "humidity": 38, "sea_level": 1010, "grnd_level": 810}, "visibility": 10000, "wind": {"speed": 2.21, "deg": 252, "gust": 3.89}, "clouds": {"all": 41}, "dt": 1715835556, "sys": {"country": "CN", "sunrise": 1715812399, "sunset": 1715860689}, "timezone": 28800, "id": 1814093, "name": "Dali", "cod": 200}我现在知道了大理的温度是25.68摄氏度，接下来我需要查询北京的温度。
Action: get_weather
Action Input: BeiJing{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 24.94, "feels_like": 23.7, "temp_min": 24.94, "temp_max": 24.94, "pressure": 1004, "hum

{'input': '大理和北京今天的温差多大？', 'output': '大理和北京今天的温差是0.74摄氏度。'}

## 7. XML Agent案例演示

#### 7.1 介绍及用途

LangChain提供多种Agent实现策略：[https://python.langchain.com/v0.1/docs/modules/agents/agent_types/](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/)

XML Agent是一种比较独特的Agent，它的特点的是在Agent策略执行过程中，中间结果使用XML而不是Json。这样对于更擅长解析XML的大模型更加友好。

例如：

* OpenAI更擅长解析Json
* Claude更擅长解析XML

文档：[https://python.langchain.com/v0.1/docs/modules/agents/agent_types/xml_agent/](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/xml_agent/)

### 7.2 使用例子

#### (1) 导入依赖

In [ ]:
! pip install langchain_anthropic

In [1]:
from langchain import hub
from langchain.agents import AgentExecutor, create_xml_agent

#### (2) 构建工具库

In [2]:
open_weather_key = "5c939a7cc59eb8696f4cd77bf75c5a9a"

In [3]:
from langchain_core.tools import tool

import requests
import json

@tool
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询上海市天气，则loc参数需要输入'ShangHai'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

In [4]:
tools = [get_weather]

#### (3) 导入提示词模版

In [5]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/xml-agent-convo")

#### (4) 将工具库绑定给大模型

In [6]:
from langchain_openai import ChatOpenAI
import openai

openai.api_key=os.getenv("OPENAI_API_KEY")
openai.api_base="https://api.openai.com/v1"
## 推荐使用 claude-3模型
llm = ChatOpenAI(model_name="gpt-3.5-turbo",api_key=openai.api_key ,base_url=openai.api_base)

#### (5) 构造Agent实例

In [7]:
# Construct the XML agent
agent = create_xml_agent(llm, tools, prompt)

#### (6) 构造AgentExecutor实例

In [8]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

#### (7) 测试

In [9]:
agent_executor.invoke({"input": "北京今天的天气怎么样?"})



> Entering new AgentExecutor chain...
<tool>get_weather</tool><tool_input>Beijing{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 800, "main": "Clear", "description": "\u6674", "icon": "01d"}], "base": "stations", "main": {"temp": 28.94, "feels_like": 27.62, "temp_min": 28.94, "temp_max": 28.94, "pressure": 1006, "humidity": 26, "sea_level": 1006, "grnd_level": 1000}, "visibility": 10000, "wind": {"speed": 3.95, "deg": 210, "gust": 5.78}, "clouds": {"all": 0}, "dt": 1716356645, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1716324830, "sunset": 1716377318}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}<final_answer>北京今天的天气是晴天，气温为28.94摄氏度。</final_answer>

> Finished chain.


{'input': '北京今天的天气怎么样?', 'output': '北京今天的天气是晴天，气温为28.94摄氏度。'}

## 8. 总结

综上，大家能够理解到的是：通过`Chains`模块，可以创建预定义的工具使用顺序。当我们知道任何用户输入所需的工具使用的具体顺序时，链就非常有用。但对于某些用例，我们使用工具的次数取决于输入。在这些情况下，我们希望让大模型本身决定使用工具的次数和顺序。这就是构建应用的第二种方式：`Agents`。

在LangChain框架中构建应用和使用外部工具的方式基本就是这两种：链式和代理式。我们可以定义为：`Chains`是一种硬编码，而`Agents`是灵活的。其相同点都是一步一步地完成一系列子任务，最终实现复杂任务。两者间的差别在于：`chains` 是提前定义好的，而对于`Agents`，当我们给了它一堆工具，不同的问题，任务的顺序可能是不一样，它会自己来拆解并完成。所以我们说`Agents`是大脑，`Chains`仅仅是借助大模型做的流程。但`Agents`的构建过程，更多依赖的是大模型的原生能力，因为其内部处理逻辑完全是由大模型本身来分析决定的，这一过程中并没有人为的介入，所以当大家构建`Agents`时如果使用的大模型本身能力不够，就需要去考虑用怎样的提示结构去引导，亦或采取微调的方式去增强模型的推理和执行能力，这些都是需要大家根据自己的实际情况进行灵活的处理。

当然，除了OpenAI的 Function Calling，及我们介绍的ReAct，还有很多`Agents`的开发范式，比如self-ask，其论文地址：https://arxiv.org/pdf/2210.03350.pdf

Agents项目中比较典型的像langchain，AutoGPT、huginn等还是非常多的，大家可以根据自己的开发需求进行相关的扩展探索。

> AutoGPT：https://github.com/Significant-Gravitas/AutoGPT

> huginn : https://github.com/huginn/huginn

## 9. Langchain 0.2转向Lang Graph

文档： [https://python.langchain.com/v0.2/docs/how_to/#agents](https://python.langchain.com/v0.2/docs/how_to/#agents)

Agents: For in depth how-to guides for agents, please check out LangGraph documentation.

[How to: use legacy LangChain Agents (AgentExecutor)](https://python.langchain.com/v0.2/docs/how_to/agent_executor/)

[How to: migrate from legacy LangChain agents to LangGraph](https://python.langchain.com/v0.2/docs/how_to/migrate_agent/)